<h1 align="center"> Computación Cientifica - Laboratorio 5: Sistemas de Reacción-Difusión </h1>
<br>
<br>
** Autores **:
+ Yerson Escobar - 201273084-8 
+ Eduardo Ramírez - 201103031-1


## Introducción



## Desarrollo

### Diferencias en 2D

a) Sea el dominio de las funciones $ u $ y $ v $: $ [a,b] \times [c,d] \times [0, T_{max}] $. <br>
Discretizaremos el dominio de todos los dominios en $ N_x + 1 $, $ N_y + 1 $, $ N_t + 1 $, tal que $ \Delta x = \frac{b - a}{N_x} $, $ \Delta y = \frac{d - c}{N_y} $, $ \Delta t = \frac{T_{max}}{N_t} $ y $ x_i = i\Delta x + a $, $ y_j = j\Delta y + c $ y $ t_k = k\Delta t $; $ (i,j,k) \in \left\lbrace 0,1,\ldots, N_x + 1\right\rbrace \times \left\lbrace 0,1,\ldots, N_y + 1 \right\rbrace \times \left\lbrace 0,1,\ldots, N_t + 1\right\rbrace $. Usando la discretización de $ u_{xx} = \frac{u_{i-1,j,k} - 2u_{i,j,k} + u_{i+1,j,k}}{\Delta x^2} $, misma para $ u_{yy} $ y $ u_t = \frac{u_{i,j,k+1} - u_{i,j,k}}{\Delta t} $ con $ u_{i,j,k} = u(x_i,y_j,t_k) $ y definiendo $\phi_{u,x} = \frac{D_u\Delta t}{\Delta x^2}; \phi_{u,y} = \frac{D_u\Delta t}{\Delta y^2}; \phi_{v,x} = \frac{D_v \Delta t}{\tau\Delta x^2}; \phi_{v,y} = \frac{D_v\Delta t}{\tau\Delta y^2}$, $U_k$ y $V_k$ matrices de soluciones en el instante $k$,  $U_k\{m\}$ es la matriz de soluciones con cada coeficiente elevado a $m$ y además:

$$
A=
\left(
\begin{array}{ccccc}
 -2 & 1 & 0 & \cdots & 1 \\
 1 & -2 & 1 & \cdots & 0 \\
 \vdots & \vdots & \ddots & \vdots & \vdots \\
 0 & \cdots & 1 & -2 & 1 \\
 1 & \cdots & 0 & 1 & -2 \\
\end{array}
\right)
$$
<br> <br>

 entonces, las EDPs discretizadas tendrán la siguiente forma: <br> <br>
$$ 
    \begin{align} 
    (1) \frac{u_{i,j,k+1} - u_{i,j,k}}{\Delta t} &= D_u \left( \frac{u_{i-1,j,k} - 2 u_{i,j,k} + u_{i+1,j,k}}{\Delta x^2} + \frac{u_{i,j-1,k} - 2u_{i,j,k} + u_{i,j+1,k}}{\Delta y^2} \right) + \lambda u_{i,j,k} - u^3_{i,j,k} - \kappa - \sigma v_{i,j,k}\\
    u_{i,j,k+1} &= [\phi_{u,x} u_{i+1,j,k} -2\phi_{u,x}u_{i,j,k} + \phi_{u,x} u_{i-1,j,k}] + [\phi_{u,y} u_{i,j+1,k} -2\phi_{u,y}u_{i,j,k} +\phi_{u,y} u_{i,j-1,k}]+ (1 +\lambda \Delta t) u_{i,j,k}  - u^3_{i,j,k} \Delta t - \kappa \Delta t - \sigma v_{i,j,k} \Delta t \\
    U_{k+1} &= A (\phi_{u,x}U_k + \phi_{u,y}U^T_k) + (\Delta t \lambda + 1)U_k - U_k\{3\} - \Delta t\kappa I - \Delta t\sigma V_k  
\end{align} 
$$
<br>

<br>
$$ 
    \begin{align}
    (2) \tau \frac{v_{i,j,k+1} - v_{i,j,k}}{\Delta t} &= D_v \left( \frac{v_{i-1,j,k} - 2 v_{i,j,k} + v_{i+1,j,k}}{\Delta x^2} + \frac{v_{i,j-1,k} - 2v_{i,j,k} + v_{i,j+1,k}}{\Delta y^2} \right) + u_{i,j,k} - v_{i,j,k} \\
        v_{i,j,k+1} &= \frac{1}{\tau}[\phi_{v,x} v_{i+1,j,k} -2\phi_{v,x}v_{i,j,k} + \phi_{v,x} v_{i-1,j,k}] + \frac{1}{\tau}[\phi_{v,y} v_{i,j+1,k} -2\phi_{v,y}v_{i,j,k} +\phi_{v,y} v_{i,j-1,k}] + \frac{\Delta t}{\tau}u_{i,j,k} + \left(1-\frac{\Delta t}{\tau}\right)v_{i,j,k} \\
        V_{k+1} &= A (\phi_{v,x}V_k + \phi_{v,y}V^T_k) +\frac{\Delta t}{\tau}U_k+ \left(\frac{1-\Delta t}{\tau}\right)V_k    
    \end{align}
$$
    
El error de aproximación es $O(\Delta t) + O((\Delta x)^2)$

b)

In [23]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import animation

def animate_pattern(sol, time_step=10, frame_step=1):
    #time index
    t_ind = 0
    fig = plt.figure()
    im = plt.imshow(sol[t_ind], cmap=plt.cm.winter)
    plt.xticks([]); plt.yticks([])

    #setting the number of frames
    frames = int(sol.shape[0]/frame_step)

    #update function
    def updatefig(t_ind):
        im = plt.imshow(sol[t_ind*frame_step], cmap=plt.cm.winter)
        return im,

    #animate it!
    ani = animation.FuncAnimation(fig, 
                                  updatefig, 
                                  frames=frames, 
                                  interval=time_step, 
                                  blit=True, 
                                  repeat=False)
    plt.show()
    
def solve_forward(u0,v0,dx,dy,dt,Tf,Du,Dv,Lambda,Tau,Sigma,Kappa):
    #----constantes----
    phiux = 1.0*Du*dt/dx**2
    print "phiux = " + str(phiux)
    phiuy = 1.0*Du*dt/dy**2
    print "phiuy = " + str(phiuy)
    phivx = 1.0*Dv*dt/(Tau*dx**2)
    print "phivx = " + str(phivx)
    phivy = 1.0*Dv*dt/(Tau*dy**2)
    print "phivy = " + str(phivy)
    if phiux >= 0.5 or phiuy >= 0.5 or phivx >= 0.5 or phivy >= 0.5:
        raise Exception("Inestable")
    Nx = u0.shape[0]-1
    Ny = u0.shape[1]-1
    ts = np.arange(0,Tf+dt,dt)
    if(ts[-1]>Tf):
        ts = ts[:-1]
    Nt = ts.size-1
    #----matrices----
    diag = -2*np.ones(Nx+1)
    A = np.diag(np.ones(Nx),-1) + np.diag(diag) + np.diag(np.ones(Nx),1)
    A[0,-1] = 1.
    A[-1,0] = 1.
    I = np.identity(Nx+1)
    u = np.zeros((Nt+1,Nx+1,Ny+1))
    v = np.zeros((Nt+1,Nx+1,Ny+1))
    u[0] = u0
    v[0] = v0
    for k in range(0,Nt):
        u[k+1] = np.dot(A,phiux*u[k] + phiuy*np.transpose(u[k])) + (dt*Lambda* + 1)*u[k] - u[k]**3 - dt*Kappa*I- dt*Sigma*v[k]
        v[k+1] = np.dot(A,phivx*v[k] + phivy*np.transpose(v[k])) + (dt/Tau)*u[k] + ((1. - dt)/Tau)*v[k]
    return u,v

u0 = np.random.rand(3,3)
v0 = np.random.rand(3,3)
dx = 0.2
dy = 0.2
dt = 0.004
Tf = 0.01
Du = 0.1#3.*np.e-4
Dv = 0.02#4.*np.e-3
Lambda = 1.0
Tau = 0.2
Sigma = 1.0
Kappa = -0.004
u,v = solve_forward(u0,v0,dx,dy,dt,Tf,Du,Dv,Lambda,Tau,Sigma,Kappa)
print u.shape
animate_pattern(u)

phiux = 0.01
phiuy = 0.01
phivx = 0.01
phivy = 0.01
(3L, 3L, 3L)


a) Sea el dominio de las funciones $ u $ y $ v $: $ [a,b] \times [c,d] \times [0, T_{max}] $. <br>
Discretizaremos el dominio de todos los dominios en $ N_x + 1 $, $ N_y + 1 $, $ N_t + 1 $, tal que $ \Delta x = \frac{b - a}{N_x} $, $ \Delta y = \frac{d - c}{N_y} $, $ \Delta t = \frac{T_{max}}{N_t} $ y $ x_i = i\Delta x + a $, $ y_j = j\Delta y + c $ y $ t_k = k\Delta t $; $ (i,j,k) \in \left\lbrace 0,1,\ldots, N_x + 1\right\rbrace \times \left\lbrace 0,1,\ldots, N_y + 1 \right\rbrace \times \left\lbrace 0,1,\ldots, N_t + 1\right\rbrace $. Usando la discretización de $ u_{xx} = \frac{u_{i-1,j,k} - 2u_{i,j,k} + u_{i+1,j,k}}{\Delta x^2} $, misma para $ u_{yy} $ y $ u_t = \frac{u_{i,j,k} - u_{i,j,k-1}}{\Delta t} $ con $ u_{i,j,k} = u(x_i,y_j,t_k) $ y definiendo $\phi_{u,x} = \frac{D_u\Delta t}{\Delta x^2}; \phi_{u,y} = \frac{D_u\Delta t}{\Delta y^2}; \phi_{v,x} = \frac{D_v \Delta t}{\tau\Delta x^2}; \phi_{v,y} = \frac{D_v\Delta t}{\tau\Delta y^2}$, $U_k$ y $V_k$ matrices de soluciones en el instante $k$,  $U_k\{m\}$ es la matriz de soluciones con cada coeficiente elevado a $m$ y además:

$$
A=
\left(
\begin{array}{ccccc}
 -2 & 1 & 0 & \cdots & 1 \\
 1 & -2 & 1 & \cdots & 0 \\
 \vdots & \vdots & \ddots & \vdots & \vdots \\
 0 & \cdots & 1 & -2 & 1 \\
 1 & \cdots & 0 & 1 & -2 \\
\end{array}
\right)
$$
<br> <br>

 entonces, las EDPs discretizadas tendrán la siguiente forma

$$ 
    \begin{align} 
    (1) \frac{u_{i,j,k} - u_{i,j,k-1}}{\Delta t} &= D_u \left( \frac{u_{i-1,j,k} - 2 u_{i,j,k} + u_{i+1,j,k}}{\Delta x^2} + \frac{u_{i,j-1,k} - 2u_{i,j,k} + u_{i,j+1,k}}{\Delta y^2} \right) + \lambda u_{i,j,k} - u^3_{i,j,k} - \kappa - \sigma v_{i,j,k}\\
        u_{i,j,k+1} - [\phi_{u,x} u_{i+1,j,k+1} -2\phi_{u,x}u_{i,j,k+1} + \phi_{u,x} u_{i-1,j,k+1}] &- [\phi_{u,y} u_{i,j+1,k+1} -2\phi_{u,y}u_{i,j,k+1} +\phi_{u,y} u_{i,j-1,k+1}] -  (1 +\lambda \Delta t) u_{i,j,k+1} + u^3_{i,j,k+1} \Delta t + \sigma v_{i,j,k+1} \Delta t = u_{i,j,k} - \kappa \Delta t\\
    \end{align}
$$
<br>

<br>
$$ 
    \begin{align}
    (2) \tau \frac{v_{i,j,k} - v_{i,j,k-1}}{\Delta t} &= D_v \left( \frac{v_{i-1,j,k} - 2 v_{i,j,k} + v_{i+1,j,k}}{\Delta x^2} + \frac{v_{i,j-1,k} - 2v_{i,j,k} + v_{i,j+1,k}}{\Delta y^2} \right) + u_{i,j,k} - v_{i,j,k} \\
        v_{i,j,k+1} - \frac{1}{\tau}[\phi_{v,x} v_{i+1,j,k+1} &- 2\phi_{v,x}v_{i,j,k+1} + \phi_{v,x} v_{i-1,j,k+1}] - \frac{1}{\tau}[\phi_{v,y} v_{i,j+1,k+1} -2\phi_{v,y}v_{i,j,k+1} +\phi_{v,y} v_{i,j-1,k+1}] - \frac{\Delta t}{\tau}u_{i,j,k+1} - \left(1-\frac{\Delta t}{\tau}\right)v_{i,j,k+1} = v_{i,j,k}
    \end{align}
$$


In [3]:
def poop_implicit_solver(u0,v0,dx,dy,dt,Tf,Du,Dv,Lambda,Tau,Sigma,Kappa)

'''
def poop_solver(u0,v0,dx,dy,dt,tf,D_u,D_v,lamb,tau,sigma,kappa):
    phiu1 = 1.0*D_u*dt/dx**2
    phiu2 = 1.0*D_u*dt/dy**2
    phiv1 = 1.0*D_v*dt/tau*dx**2
    phiv2 = 1.0*D_v*dt/tau*dy**2
    if phiu1 >= 0.5 or phiu2 >= 0.5 or phiv1 >= 0.5 or phiv2 >= 0.5:
        raise Exception("Inestable")
    Nx = u0.shape[0] - 1
    Ny = u0.shape[1] - 1
    ts = np.arange(0,Tmax,dt)
    Nt = ts.shape[0] - 1
    u = np.zeros([Nt+1, Nx+1, Ny+1])
    v = np.zeros([Nt+1, Nx+1, Ny+1])
    
    #pa formar las matrices con las constantes
    
    dim = Nx > Ny if Nx else Ny
    matrix_phiu1 = np.zeros([dim + 1, dim + 1])
    matrix_phiu2 = np.zeros([dim + 1, dim + 1])
    matrix_phiv1 = np.zeros([dim + 1, dim + 1])
    matrix_phiv2 = np.zeros([dim + 1, dim + 1])
    
    matrix_phiu1[0,0] = 1. - 2.*phiu1
    matrix_phiu1[dim,dim] = matrix_phiu1[0,0]
    matrix_phiu1[0,1] = phiu1
    matrix_phiu1[dim,dim-1] = matrix_phiu1[0,1]
    matrix_phiu1[0,dim] = phiu1
    matrix_phiu1[dim,0] = matrix_phiu1[0,Nx]
    matrix_phiu1[i-1:i+2 for i in range(1,dim-1)] = np.array([phiu1, 1. - 2.*phiu1, phiu1])
    
    matrix_phiu2[0,0] = -lamb*dt - 2.*phiu2
    matrix_phiu2[dim,dim] = matrix_phiu2[0,0]
    matrix_phiu2[0,1] = phiu2
    matrix_phiu2[dim,dim-1] = matrix_phiu2[0,1]
    matrix_phiu2[0,dim] = phiu2
    matrix_phiu2[dim,0] = matrix_phiu2[0,dim]
    matrix_phiu2[i-1:i+2 for i in range(1,dim-1)] = np.array([phiu2, -lamb*dt. - 2.*phiu2, phiu2])
    
    matrix_phiv1[0,0] = 1. - 2.*phiv1
    matrix_phiv1[dim,dim] = matrix_phiv1[0,0]
    matrix_phiv1[0,1] = phiv1
    matrix_phiv1[dim,dim-1] = matrix_phiv1[0,1]
    matrix_phiv1[0,dim] = phiv1
    matrix_phiv1[dim,0] = matrix_phiv1[0,dim]
    matrix_phiv1[i-1:i+2 for i in range(1,dim-1)] = np.array([phiv1, -1. - 2.*phiv1, phiv1])
    
    matrix_phiv2[0,0] = -dt/phiv2 - 1.*dt/tau
    matrix_phiv2[dim,dim] = matrix_phiv2[0,0]
    matrix_phiv2[0,1] = phiv2
    matrix_phiv2[dim,dim-1] = matrix_phiv2[0,1]
    matrix_phiv2[0,dim] = phiv2
    matrix_phiv2[dim,0] = matrix_phiv2[0,dim]
    matrix_phiv2[i-1:i+2 for i in range(1,dim-1)] = np.array([phiv2, -dt/phiv2 - 1.*dt/tau, phiv2])
    
    
    for t in ts:
        if t == 0:
            u[0] = u0
            v[0] = v0
        else:
            u[t] = np.dot(matrix_phiu1,u[k-1]) + np.dot(np.transpose(u[k-1]),np.transpose(matrix_phiu2)) + np.power(u[k-1], 3)*dt - kappa*dt*np.ones([Nx+1,Ny+1]) - sigma*v[0]
            v[t] = np.dot(matrix_phiv1,v[k-1]) + np.dot(np.transpose(v[k-1]),np.transpose(matrix_phiv2)) + (dt/tau)*u[k-1]
    
    return u, v
'''
A = np.diag(np.ones(3),-1)
print A

[[ 0.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]]


### Método de las Líneas

Sea el dominio de las funciones $ u $ y $ v $: $ [a,b] \times [c,d] \times [0, T_{max}] $. <br>
Discretizaremos el dominio espacial en $ N_x + 1 $, $ N_y + 1 $ tal que $ \Delta x = \frac{b - a}{N_x} $, $ \Delta y = \frac{d - c}{N_y} $y $ x_i = i\Delta x + a $, $ y_j = j\Delta y + c $ , $ (i,j) \in \left\lbrace 0,1,\ldots, N_x + 1\right\rbrace \times \left\lbrace 0,1,\ldots, N_y + 1 \right\rbrace $. Usando la discretización de $ u_{xx} = \frac{u_{i-1,j} - 2u_{i,j} + u_{i+1,j}}{\Delta x^2} $, misma para $ u_{yy} $ y usando la derivada temporal como $\frac{\partial u_{i,j}}{\partial t}$ y definiendo $\phi_{u,x} = \frac{D_u}{\Delta x^2}; \phi_{u,y} = \frac{D_u}{\Delta y^2}; \phi_{v,x} = \frac{D_v}{\tau\Delta x^2}; \phi_{v,y} = \frac{D_v}{\tau\Delta y^2}$, $U$ y $V$ matrices de soluciones en algun instante,  $U_k\{m\}$ es la matriz de soluciones con cada coeficiente elevado a $m$, $DU$ y $DV$ son las matrices de derivadas temporales en todos los puntos $(i,j)$ y además:
$$
A=
\left(
\begin{array}{ccccc}
 -2 & 1 & 0 & \cdots & 1 \\
 1 & -2 & 1 & \cdots & 0 \\
 \vdots & \vdots & \ddots & \vdots & \vdots \\
 0 & \cdots & 1 & -2 & 1 \\
 1 & \cdots & 0 & 1 & -2 \\
\end{array}
\right)
$$
<br> <br>

 entonces, las EDPs discretizadas tendrán la siguiente forma
$$
\begin{align} 
    (1) \frac{\partial u_{i,j}}{\partial t} &= D_u \left( \frac{u_{i-1,j} - 2 u_{i,j} + u_{i+1,j}}{\Delta x^2} + \frac{u_{i,j-1} - 2u_{i,j} + u_{i,j+1}}{\Delta y^2} \right) + \lambda u_{i,j} - u^3_{i,j} - \kappa - \sigma v_{i,j}\\
    \frac{\partial u_{i,j}}{\partial t} &= [\phi_{u,x} u_{i+1,j} -2\phi_{u,x}u_{i,j} + \phi_{u,x} u_{i-1,j}] + [\phi_{u,y} u_{i,j+1} -2\phi_{u,y}u_{i,j} +\phi_{u,y} u_{i,j-1}]+ \lambda u_{i,j}  - u^3_{i,j} - \kappa  - \sigma v_{i,j}  \\
    DU &= A (\phi_{u,x}U + \phi_{u,y}U^T) + \lambda U- U\{3\} - \kappa I - \sigma V  
\end{align} 
$$

$$
\begin{align} 
    (2) \frac{\partial v_{i,j}}{\partial t} &= D_v \left( \frac{v_{i-1,j} - 2 v_{i,j} + v_{i+1,j}}{\Delta x^2} + \frac{v_{i,j-1} - 2v_{i,j} + v_{i,j+1}}{\Delta y^2} \right) + \frac{1}{\tau}u_{i,j} - \frac{1}{\tau}v_{i,j}\\
    \frac{\partial v_{i,j}}{\partial t} &= [\phi_{v,x} v_{i+1,j} -2\phi_{v,x}v_{i,j} + \phi_{v,x} v_{i-1,j}] + [\phi_{v,y} v_{i,j+1} -2\phi_{v,y}v_{i,j} +\phi_{v,y} u_{i,j-1}] + \frac{1}{\tau}v_{i,j} - \frac{1}{\tau}v_{i,j} \\
    DV &= A (\phi_{v,x}U + \phi_{v,y}U^T) + \frac{1}{\tau}U - \frac{1}{\tau}V
\end{align} 
$$

In [ ]:
def solve_mol(u0,v0,dx,dy,dt,Tf,Du,Dv,Lambda,Tau,Sigma,Kappa,method):
    #----constantes----
    phiux = 1.0*Du/dx**2
    print "phiux = " + str(phiux)
    phiuy = 1.0*Du/dy**2
    print "phiuy = " + str(phiuy)
    phivx = 1.0*Dv/(Tau*dx**2)
    print "phivx = " + str(phivx)
    phivy = 1.0*Dv/(Tau*dy**2)
    print "phivy = " + str(phivy)
    if phiux >= 0.5 or phiuy >= 0.5 or phivx >= 0.5 or phivy >= 0.5:
        raise Exception("Inestable")
    Nx = u0.shape[0]-1
    Ny = u0.shape[0]-1
    ts = np.arange(0,Tf+dt,dt)
    if(ts[-1]>Tf):
        ts = ts[:-1]
    Nt = ts.size-1
    #----matrices----
    ones = np.ones(Nx)
    diag = -2*np.ones(Nx+1)
    A = np.diag(ones,-1) + np.diag(diag) + np.diag(ones,1)
    A[0,-1] = 1.
    A[-1,0] = 1.
    I = np.identity(Nx+1)
    u = np.zeros((Nt+1,Nx+1,Ny+1))
    v = np.zeros((Nt+1,Nx+1,Ny+1))
    u[0] = u0
    v[0] = v0
    if method == "RK2":
      for k in range(0,Nt):
        uk = np.dot(A,phiux*u[k] + phiuy*np.transpose(u[k])) + Lambda*u[k] - u[k]**3 -Kappa*I-Sigma*v[k]
        k1u = uk
        k2u = uk + (dt/2.0)*k1u
        k3u = uk + (dt/2.0)+k2u
        k4u = uk + k3u*k2u
        u[k+1] = uk + (k/6.0)*(k1u + 2*k2u +)
        
        v[k+1] = np.dot(A,phivx*v[k] + phivy*np.transpose(v[k])) + (dt/Tau)*u[k] + ((1. - dt)/Tau)*v[k]
    elif method == "RK4"
    
    else:
        raise Exception("Method not RK2 or RK4")
    